# Домашнее задание №1: линейная регрессия и векторное дифференцирование (10 баллов).

* Максимальное количество баллов за задания в ноутбуке - 11, но больше 10 оценка не ставится, поэтому для получения максимальной оценки можно сделать не все задания.

* Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [9]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [100]:
len("Сорокин")%4+1

4

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [34]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

1.5771471960728363e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [71]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.000000000000000000000000000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

2.565379847746644e-25


array([ 1.10036474e-14,  8.43621277e-15,  4.17617954e-16, -8.10587165e-15,
        5.02894297e-15, -7.68236281e-15,  1.01001437e-14, -1.50357560e-15,
        6.64261501e+00,  4.93498553e+01,  1.56484208e-14,  1.56711303e-14,
        1.41586298e-15,  1.29999428e-14, -1.22470142e-14, -8.93073356e-15,
       -1.01589318e-14,  9.26671597e-15, -9.74873713e-15,  3.66928158e+01,
        3.00457976e+01,  9.91948104e-16, -1.42840666e-15,  4.33888362e+01,
       -1.30171834e-15, -8.59023113e-15,  1.62074799e-14,  5.06469368e+01,
       -1.23297225e-14,  1.03669252e-14, -5.34225705e-15,  6.35232542e-15,
        1.09415394e-15,  4.36546915e-15, -1.71696989e-14, -3.68106056e-15,
        3.24951308e-15, -1.08654180e-14, -1.03834351e-15, -2.26219410e-15,
       -5.26785711e-15, -2.14869240e-16,  3.22143469e-15,  5.66681142e+01,
       -7.37251700e-15,  8.17237113e-15, -1.07395499e-15,  9.10309651e-15,
        5.36839551e-16,  8.53728735e-15, -6.13509604e-15,  1.60969197e-14,
        5.99152058e-15,  

***Задание 1 (0.5 балла).*** Объясните, чем вызвано различие двух полученных значений метрики?

***Задание 2 (0.5 балла).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

**Ответ на задание 1** Linear regression решает задачу обучения линейной регрессии аналитически (формула: $\frac{1}{l} ||X\omega - y ||^2 \rightarrow min$). Метод SGDRegressor это stochastic gradient decent. Стохастическая история достаточно быстрая но менее точная по сравнению со своими прямым методом, связано это с тем что на каждой итерации SGDReg берет случайный параметр для оценки весов 

**Ответ на задание 2** Alpha=0.000000000000000000000000000001 а MSE=2.565379847746644e-25 (делал в коде парой строк выше)

## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс. 

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и второй варианты) или l2 (третий и четвертый варианты) регуляризацию. 

In [101]:
class LinearRegression(object):
    def __init__(self, alpha=0.0001, l_ratio=0.001, tol=0.001, max_iter=1000):
        '''
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения 
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        
        # your code here
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
             
    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''

        self.w = np.zeros(X.shape[1])
        self.errors = []

        for i in range(self.max_iter):
            y_pred = X.dot(self.w)
            mse_error = np.mean(np.square(X.dot(self.w) - y)) / 2.0
            self.errors.append(mse_error)

            grad = np.dot(X.T, (y_pred-y)) / len(X)
            self.w = self.w - self.alpha*grad
            if abs(mse_error - grad) < self.tol:
                 break

    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        # your code here
        y_pred = X @ self.w
        return  y_pred

In [102]:
my_reg = LinearRegression()
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки

* с l1-регуляризацией (from sklearn.linear_model import Lasso, **первый и второй вариант**) или с l2-регуляризацией (from sklearn.linear_model import Ridge, **третий и четвертый вариант**)
* со значением параметра регуляризации **0.1 - для первого и третьего варианта, 0.01 - для второго и четвертого варианта**. 

Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [96]:
#your code here
np.random.seed(228)
my_reg_ridge = LinearRegression(alpha = 0.01, l_ratio = 0.1)
my_reg_ridge.fit(X, y)
mean_squared_error(y, my_reg_ridge.predict(X))

0.0001610627431129351

In [98]:
np.random.seed(228)
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.01)
ridge.fit(X, y)

print(mean_squared_error(y, ridge.predict(X)))

3.4159798606448106e-08


***Задание 6* (1 балл).***
Пусть $P, Q \in \mathbb{R}^{n\times n}$. Найдите $\nabla_Q tr(PQ)$

***Задание 7* (1 балл).***
Пусть $x, y \in \mathbb{R}^{n}, M \in \mathbb{R}^{n\times n}$. Найдите $\nabla_M x^T M y$

Решения заданий 6 и 7 можно написать на листочке и отправить в anytask вместе с заполненным ноутбуком.